#### testing

In [5]:
from load_data import *

In [6]:
# load_data()

## Loading the data

In [7]:
from load_data import *

In [8]:
X_train,X_test,y_train,y_test = load_data()

OpenCV(4.5.1) /tmp/pip-req-build-hj027r8z/opencv/modules/imgproc/src/resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

./data/data/Shine/shine131.jpg
OpenCV(4.5.1) /tmp/pip-req-build-hj027r8z/opencv/modules/imgproc/src/resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

./data/data/Rain/rain141.jpg


In [9]:
len(X_train),len(y_train)

(843, 843)

In [10]:
len(X_test),len(y_test)

(280, 280)

## Test Modelling

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [12]:
class Test_Model(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.c1 = nn.Conv2d(1,32,5)
        self.c2 = nn.Conv2d(32,64,5)
        self.c3 = nn.Conv2d(64,128,5)
        self.fc4 = nn.Linear(128*21*21,256)
        self.fc5 = nn.Linear(256,4)
    
    def forward(self,X):
        preds = F.max_pool2d(F.relu(self.c1(X)))
        preds = F.max_pool2d(F.relu(self.c1(preds)))
        preds = F.max_pool2d(F.relu(self.c1(preds)))
        print(preds.shape)
#         preds = preds.view(-1,1*1524)
#         preds = F.relu(self.fc4(preds))
#         preds = self.fc5(preds)
        return preds

In [13]:
device = torch.device('cuda')

In [14]:
BATCH_SIZE = 32

In [15]:
IMG_SIZE = 112

In [16]:
model = Test_Model().to(device)

In [17]:
optimizer = optim.SGD(model.parameters(),lr=0.1)

In [18]:
criterion = nn.CrossEntropyLoss()

In [19]:
EPOCHS = 12

In [20]:
from tqdm import tqdm

In [21]:
PROJECT_NAME = 'Weather-Clf'

In [22]:
test_index = 0

In [ ]:
wandb.init(project=PROJECT_NAME,name=f'test-{test_index}')
for _ in tqdm(range(EPOCHS)):
    for i in range(0,len(X_train),BATCH_SIZE):
        X_batch = X_train[i:i+BATCH_SIZE].view(-1,1,112,112).to(device)
        y_batch = y_train[i:i+BATCH_SIZE].to(device)
        model.to(device)
        preds = model(X_batch.float())
        preds.to(device)
        loss = criterion(preds,torch.tensor(y_batch,dtype=torch.long))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        wandb.log({'loss':loss.item})
test_index += 1